In [1]:
import pyspark

from pyspark.sql import SparkSession
# import findspark
# findspark.init()pip

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
os.getcwd()

'/Users/firaterman/Documents/git_workspace/FERMAN/pyspark'

In [ ]:
#Define the sparksession

spark = SparkSession.builder.appName("pp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 17:29:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# import pfile 
df = spark.read.option("header","true").csv("/home/ferman/Documents/data/NameAge.csv")
df.show()


NameError: name 'spark' is not defined

In [5]:
df.printSchema()

NameError: name 'df' is not defined

In [6]:
df.select("name").show()

NameError: name 'df' is not defined

In [82]:
df.select(["name","age"]).show()

+------+---+
|  name|age|
+------+---+
| prash| 35|
|Serhat| 28|
| roxto| 45|
| Malim| 25|
+------+---+



In [83]:
type(df.select(["name","age"]))

pyspark.sql.dataframe.DataFrame

In [84]:
df.dtypes

[('name', 'string'), ('age', 'string')]

In [85]:
#check if we have a dataframe and format of columns
df.describe()

DataFrame[summary: string, name: string, age: string]

In [86]:
df.describe().show()

+-------+-----+----------------+
|summary| name|             age|
+-------+-----+----------------+
|  count|    4|               4|
|   mean| null|           33.25|
| stddev| null|8.88350531415762|
|    min|Malim|              25|
|    max|roxto|              45|
+-------+-----+----------------+



In [89]:
df = df.withColumn("add5years",df['age']+2)
df.show()


+------+---+---------+
|  name|age|add5years|
+------+---+---------+
| prash| 35|     37.0|
|Serhat| 28|     30.0|
| roxto| 45|     47.0|
| Malim| 25|     27.0|
+------+---+---------+



In [91]:
# drop columns
df = df.drop("add5years")
df.show()


+------+---+
|  name|age|
+------+---+
| prash| 35|
|Serhat| 28|
| roxto| 45|
| Malim| 25|
+------+---+



In [93]:
#Rename columns
df = df.withColumnRenamed("name","surname")
df.show()

+-------+---+
|surname|age|
+-------+---+
|  prash| 35|
| Serhat| 28|
|  roxto| 45|
|  Malim| 25|
+-------+---+



## MISSING VALUES 

In [7]:
# import a new dataset
dfCity = spark.read.csv("/home/ferman/Documents/data/NameAgeCity.csv", header=True, inferSchema=True)
dfCity.show()

+------+----+-----+
|  name| age| city|
+------+----+-----+
| prash|  35|Paris|
|Serhat|  28| null|
|  null|  45| Amed|
| Malim|  25|kyoto|
| Malim|null|tokyo|
+------+----+-----+



In [13]:
# drop all columns conaining null values
dfCity.na.drop().show()

+-----+---+-----+
| name|age| city|
+-----+---+-----+
|prash| 35|Paris|
|Malim| 25|kyoto|
+-----+---+-----+



In [14]:
# drop only the column with any
dfCity.na.drop(how="any").show()

+-----+---+-----+
| name|age| city|
+-----+---+-----+
|prash| 35|Paris|
|Malim| 25|kyoto|
+-----+---+-----+



In [15]:
# drop only those with whole raw containing null values
dfCity.na.drop(how="all").show()

+------+----+-----+
|  name| age| city|
+------+----+-----+
| prash|  35|Paris|
|Serhat|  28| null|
|  null|  45| Amed|
| Malim|  25|kyoto|
| Malim|null|tokyo|
+------+----+-----+



In [22]:
# drop only the column with any and treshhold means at least "x" value of the treshold is non missing values
dfCity.na.drop(how="any",thresh=3).show()

+-----+---+-----+
| name|age| city|
+-----+---+-----+
|prash| 35|Paris|
|Malim| 25|kyoto|
+-----+---+-----+



In [30]:
# drop missing values only for this column so it drop whole row
dfCity.na.drop(how="any",subset=["age"]).show()

+------+---+-----+
|  name|age| city|
+------+---+-----+
| prash| 35|Paris|
|Serhat| 28| null|
|  null| 45| Amed|
| Malim| 25|kyoto|
+------+---+-----+



In [23]:
# filling the missing value

dfCity.na.fill("MissingValue").show()

+------------+----+------------+
|        name| age|        city|
+------------+----+------------+
|       prash|  35|       Paris|
|      Serhat|  28|MissingValue|
|MissingValue|  45|        Amed|
|       Malim|  25|       kyoto|
|       Malim|null|       tokyo|
+------------+----+------------+



In [26]:
# only for specific columns
dfCity.na.fill("MissingValue",["age","city"]).show()

+------+----+------------+
|  name| age|        city|
+------+----+------------+
| prash|  35|       Paris|
|Serhat|  28|MissingValue|
|  null|  45|        Amed|
| Malim|  25|       kyoto|
| Malim|null|       tokyo|
+------+----+------------+



## Imputer replace with missing values

In [48]:
# add inferSchema to consider number as integer
dfSalary = spark.read.csv("/home/ferman/Documents/data/NameAgeCitySalary.csv", header = True, inferSchema=True)
dfSalary.show()

+------+----+-----+------+
|  name| age| city|salary|
+------+----+-----+------+
| prash|  35|Paris|  2000|
|Serhat|null| null|  1000|
|  null|  45| Amed|  null|
| Malim|  25|kyoto| 20000|
| halwo|null|tokyo|  null|
+------+----+-----+------+



In [54]:
# repalce non values by mean/average/etc...
from pyspark.ml.feature import Imputer

targetedColumns = ['age','salary']
strategy= "median" #median or else
imputer = Imputer (
    inputCols = targetedColumns,
    outputCols= ['{}_imputer'.format(c) for c in targetedColumns]

).setStrategy(strategy)

In [56]:
dfWithImputer = imputer.fit(dfSalary).transform(dfSalary)
dfWithImputer.show()

+------+----+-----+------+-----------+--------------+
|  name| age| city|salary|age_imputer|salary_imputer|
+------+----+-----+------+-----------+--------------+
| prash|  35|Paris|  2000|         35|          2000|
|Serhat|null| null|  1000|         35|          1000|
|  null|  45| Amed|  null|         45|          2000|
| Malim|  25|kyoto| 20000|         25|         20000|
| halwo|null|tokyo|  null|         35|          2000|
+------+----+-----+------+-----------+--------------+



## Filter

In [63]:
# filter
dfWithImputer.filter("salary>2000").select(["salary","age"]).show()


+------+---+
|salary|age|
+------+---+
| 20000| 25|
+------+---+



In [59]:
dfWithImputer.filter("salary>2000").show()

+-----+---+-----+------+-----------+--------------+
| name|age| city|salary|age_imputer|salary_imputer|
+-----+---+-----+------+-----------+--------------+
|Malim| 25|kyoto| 20000|         25|         20000|
+-----+---+-----+------+-----------+--------------+



#### another way to filter : BEST WAY

In [69]:
dfWithImputer.filter(dfWithImputer["salary"]>3000).show()

+-----+---+-----+------+-----------+--------------+
| name|age| city|salary|age_imputer|salary_imputer|
+-----+---+-----+------+-----------+--------------+
|Malim| 25|kyoto| 20000|         25|         20000|
+-----+---+-----+------+-----------+--------------+



In [72]:
dfWithImputer.filter((dfWithImputer["salary"]>3000) & (dfWithImputer['age']>10)).show()

+-----+---+-----+------+-----------+--------------+
| name|age| city|salary|age_imputer|salary_imputer|
+-----+---+-----+------+-----------+--------------+
|Malim| 25|kyoto| 20000|         25|         20000|
+-----+---+-----+------+-----------+--------------+



In [73]:
dfWithImputer.filter((dfWithImputer["salary"]>3000) & (dfWithImputer['age']>10)).select("age").show()

+---+
|age|
+---+
| 25|
+---+



In [ ]:
dfWithImputer